In [1]:
from predicciones import *

In [3]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"

print(text)
eval_text(text,tokenizer,model)

PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias


array([ 1,  3,  4,  4,  5,  6,  6,  7, 10,  8,  9, 10, 10])